This network is to test code done for Project 2. 
Go down to the 'full prediction' heading to check out the model in action!

In [1]:
import torch
import math
import prologue as prologue
import framework


torch.set_grad_enabled(True) #By the end we should have this set to False

In [2]:
# In this block we discuss put all the external functions, 
# namely generate_disc, computing number of errors, and training the model.
# We should not need to change any of this.

######################################################################
######################### DATA things ################################
######################################################################
def generate_disc_set(nb): # This can be kept the same
    input = torch.empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

train_input, train_target = generate_disc_set(100)
test_input, test_target = generate_disc_set(100)

# center the data:
mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

mini_batch_size = 1

######################################################################
######################### ERROR things ################################
######################################################################

def compute_nb_errors(model, data_input, data_target): #this can be kept the same

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors


######################################################################
######################### TRAIN things ################################
######################################################################
# The following should be modified

def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss() #replace this by self macde MSE
    optimizer = optim.SGD(model.parameters(), lr = 1e-1) #replace this by self made SGD for MSE
    nb_epochs = 10

    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad() # replace this so it works with 
            loss.backward() # replace this to self made backward method
            optimizer.step() # '_'



# full prediction

Now we try to do a full prediction task with one fully connected linear layer, and one relu nonlin, and MSE.

In [99]:
# A helper function. Not sure how useful it is
def modif_target(val):
    if val==0:
        return -1
    else:
        return 1

In [100]:
#this is our model. The last output is our "prediction" which we compute the loss  with.
model = framework.Sequential((framework.Linear(2, 50), 
                              framework.ReLu(), 
                              framework.Linear(50, 25), 
                              framework.ReLu(),
                              framework.Linear(25, 1)))
loss = framework.MSE() 

In [101]:
# first compute the total loss on the first inputs:
loss_acc = 0
for i in range(100):
    loss_acc += loss.forward(model.forward(train_input[i]), modif_target(train_target[i]))
print(loss_acc)

tensor([108.3281])


In [18]:
#training on the first samples. We run backward by flowing back the derivative of the loss. 
# Whenever the backward method is ran, each layer stores an updated gradient. 
for i in range(100):
    model.backward(loss.backward(model.forward(train_input[i]), modif_target(train_target[i])))

In [91]:
# then perform an optimization step. This updates the weights by using the stored gradient.
model.SGD_step(0.005)

In [92]:
#Here we compute the accuracy:
tot_right=0
for i in range(100):
    #print(model.forward(train_input[i]))
    #print(train_target[i])
    if model.forward(train_input[i]) <0.0 and  train_target[i]==0:
        tot_right+=1
    elif model.forward(train_input[i]) >=0.0 and  train_target[i]==1:
        tot_right+=1
print(tot_right)

46


In [114]:
#this is our model. The last output is our "prediction" which we compute the loss  with.
model = framework.Sequential((framework.Linear(2, 10), 
                              framework.Tanh(), 
                              framework.Linear(10, 10), 
                              framework.Tanh(),
                              framework.Linear(10, 50), 
                              framework.Tanh(),
                              framework.Linear(50, 1)))
loss = framework.MSE() 

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

nb_epochs = 100
batch_size = 50
for e in range(nb_epochs):
    loss_acc=0
    for b in range(0, 1000, batch_size):
        for i in range(batch_size):
            prediction = model.forward(train_input[b+i])
            loss_acc += loss.forward(prediction, train_target[b+i])
            model.backward(loss.backward(prediction, train_target[b+i]))
        model.SGD_step(0.0005)
    print('epcoh nb: ', e, 'loss: ', loss_acc)
        

epcoh nb:  0 loss:  tensor([342.7868])
epcoh nb:  1 loss:  tensor([264.6042])
epcoh nb:  2 loss:  tensor([263.1952])
epcoh nb:  3 loss:  tensor([262.3332])
epcoh nb:  4 loss:  tensor([261.6960])
epcoh nb:  5 loss:  tensor([261.1650])
epcoh nb:  6 loss:  tensor([260.6961])
epcoh nb:  7 loss:  tensor([260.2710])
epcoh nb:  8 loss:  tensor([259.8813])
epcoh nb:  9 loss:  tensor([259.5216])
epcoh nb:  10 loss:  tensor([259.1881])
epcoh nb:  11 loss:  tensor([258.8783])
epcoh nb:  12 loss:  tensor([258.5893])
epcoh nb:  13 loss:  tensor([258.3192])
epcoh nb:  14 loss:  tensor([258.0660])
epcoh nb:  15 loss:  tensor([257.8282])
epcoh nb:  16 loss:  tensor([257.6040])
epcoh nb:  17 loss:  tensor([257.3917])
epcoh nb:  18 loss:  tensor([257.1908])
epcoh nb:  19 loss:  tensor([256.9999])
epcoh nb:  20 loss:  tensor([256.8181])
epcoh nb:  21 loss:  tensor([256.6445])
epcoh nb:  22 loss:  tensor([256.4781])
epcoh nb:  23 loss:  tensor([256.3186])
epcoh nb:  24 loss:  tensor([256.1655])
epcoh nb: 

KeyboardInterrupt: 

In [43]:
nb_epochs = 100
batch_size = 50
for e in range(nb_epochs):
    loss_acc=0
    for b in range(0, 500, batch_size):
        for i in range(batch_size):
            prediction = model.forward(train_input[b+i])
            loss_acc += loss.forward(prediction, train_target[b+i])
            model.backward(loss.backward(prediction, train_target[b+i]))
        model.SGD_step(0.0002)
    print('epcoh nb: ', e, 'loss: ', loss_acc)

epcoh nb:  0 loss:  tensor([35.7954])
epcoh nb:  1 loss:  tensor([35.7946])
epcoh nb:  2 loss:  tensor([35.7937])
epcoh nb:  3 loss:  tensor([35.7928])
epcoh nb:  4 loss:  tensor([35.7920])
epcoh nb:  5 loss:  tensor([35.7911])
epcoh nb:  6 loss:  tensor([35.7903])
epcoh nb:  7 loss:  tensor([35.7894])
epcoh nb:  8 loss:  tensor([35.7885])
epcoh nb:  9 loss:  tensor([35.7876])
epcoh nb:  10 loss:  tensor([35.7868])
epcoh nb:  11 loss:  tensor([35.7859])
epcoh nb:  12 loss:  tensor([35.7851])
epcoh nb:  13 loss:  tensor([35.7842])
epcoh nb:  14 loss:  tensor([35.7834])
epcoh nb:  15 loss:  tensor([35.7825])
epcoh nb:  16 loss:  tensor([35.7816])
epcoh nb:  17 loss:  tensor([35.7808])
epcoh nb:  18 loss:  tensor([35.7799])
epcoh nb:  19 loss:  tensor([35.7791])
epcoh nb:  20 loss:  tensor([35.7782])
epcoh nb:  21 loss:  tensor([35.7774])
epcoh nb:  22 loss:  tensor([35.7765])
epcoh nb:  23 loss:  tensor([35.7756])
epcoh nb:  24 loss:  tensor([35.7748])
epcoh nb:  25 loss:  tensor([35.773

In [44]:
#Here we compute the accuracy:
tot_right=0
for i in range(500):
    #print(model.forward(train_input[i]))
    #print(train_target[i])
    if model.forward(test_input[i]) <0.0 and  test_target[i]==0:
        tot_right+=1
    elif model.forward(test_input[i]) >=0.0 and  test_target[i]==1:
        tot_right+=1
print(tot_right)

334


# Clean

In [131]:
######################################################################
######################### DATA things ################################
######################################################################
def generate_disc_set(nb): # This can be kept the same
    input = torch.empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target



######################################################################
######################### ERROR things ################################
######################################################################


def compute_nb_errors(model, data_input, data_target):
    tot_right = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        for i in range(mini_batch_size):
            output = model.forward(data_input[b+i])
            if output <0.0 and  data_target[b+i]==0:
                tot_right+=1
            elif output >=0.0 and  data_target[b+i]==1:
                tot_right+=1

    return tot_right/data_input.size(0)



######################################################################
######################### TRAIN things ################################
######################################################################
#Training and Testing
def train_model(model, train_input, train_target):
    nb_epochs = 250
    batch_size = 50
    for e in range(nb_epochs):
        loss_acc=0
        for b in range(0, 1000, batch_size):
            for i in range(batch_size):
                prediction = model.forward(train_input[b+i])
                loss_acc += loss.forward(prediction, train_target[b+i])
                model.backward(loss.backward(prediction, train_target[b+i]))
            model.SGD_step(0.0005)
        if e%5 == 0:
            print('epcoh nb: ', e, 'loss: ', loss_acc)



In [126]:
model = framework.Sequential((framework.Linear(2, 10), 
                              framework.Tanh(), 
                              framework.Linear(10, 10), 
                              framework.Tanh(),
                              framework.Linear(10, 50), 
                              framework.Tanh(),
                              framework.Linear(50, 1)))

loss = framework.MSE()

In [127]:
train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

# center the data:
mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)


train_model(model, train_input, train_target)

epcoh nb:  0 loss:  tensor([332.4319])
epcoh nb:  5 loss:  tensor([254.9257])
epcoh nb:  10 loss:  tensor([248.7937])
epcoh nb:  15 loss:  tensor([238.6078])
epcoh nb:  20 loss:  tensor([215.4829])
epcoh nb:  25 loss:  tensor([170.4130])
epcoh nb:  30 loss:  tensor([121.7979])
epcoh nb:  35 loss:  tensor([130.3188])
epcoh nb:  40 loss:  tensor([102.8324])
epcoh nb:  45 loss:  tensor([92.8497])
epcoh nb:  50 loss:  tensor([88.2676])
epcoh nb:  55 loss:  tensor([85.8224])
epcoh nb:  60 loss:  tensor([84.3027])
epcoh nb:  65 loss:  tensor([83.2431])
epcoh nb:  70 loss:  tensor([82.4418])
epcoh nb:  75 loss:  tensor([81.8002])
epcoh nb:  80 loss:  tensor([81.2650])
epcoh nb:  85 loss:  tensor([80.8055])
epcoh nb:  90 loss:  tensor([80.4018])
epcoh nb:  95 loss:  tensor([80.0410])
epcoh nb:  100 loss:  tensor([79.7142])
epcoh nb:  105 loss:  tensor([79.4148])
epcoh nb:  110 loss:  tensor([79.1381])
epcoh nb:  115 loss:  tensor([78.8805])
epcoh nb:  120 loss:  tensor([78.6388])
epcoh nb:  12

In [132]:
compute_nb_errors(model, test_input, test_target)



0.73

#Below this point is previous practicals ad drafs

In [6]:
#Problem Set 3:
#/usr/bin/env python

# Any copyright is dedicated to the Public Domain.
# https://creativecommons.org/publicdomain/zero/1.0/

# Written by Francois Fleuret <francois@fleuret.org>



######################################################################

def sigma(x):
    return x.tanh()

def dsigma(x):
    return 4 * (x.exp() + x.mul(-1).exp()).pow(-2)

######################################################################

def loss(v, t): #This is MSE, right?
    return (v - t).pow(2).sum()

def dloss(v, t):
    return 2 * (v - t)

######################################################################

def forward_pass(w1, b1, w2, b2, x):
    x0 = x
    s1 = w1.mv(x0) + b1
    x1 = sigma(s1)
    s2 = w2.mv(x1) + b2
    x2 = sigma(s2)

    return x0, s1, x1, s2, x2

#whose arguments correspond to the network’s parameters, 
#the target vector, the quantities computed by the forward pass, 
#and the tensors used to store the cumulated sums of the gradient 
#on individual samples, and updates the latters according to the
#formula of the backward pass.

def backward_pass(w1, b1, w2, b2, #current weights and biases of the network
                  t, # target vector
                  x, s1, x1, s2, x2, #output of network
                  dl_dw1, dl_db1, dl_dw2, dl_db2): #tensors used to stor the sumulated sums of the gradient
    x0 = x
    dl_dx2 = dloss(x2, t)
    dl_ds2 = dsigma(s2) * dl_dx2
    dl_dx1 = w2.t().mv(dl_ds2)
    dl_ds1 = dsigma(s1) * dl_dx1

    dl_dw2.add_(dl_ds2.view(-1, 1).mm(x1.view(1, -1)))
    dl_db2.add_(dl_ds2)
    dl_dw1.add_(dl_ds1.view(-1, 1).mm(x0.view(1, -1)))
    dl_db1.add_(dl_ds1)

######################################################################


In [8]:
nb_classes = train_target.size(1)
nb_train_samples = train_input.size(0)

zeta = 0.90

train_target = train_target * zeta
test_target = test_target * zeta

nb_hidden = 50
eta = 1e-1 / nb_train_samples
epsilon = 1e-6

w1 = torch.empty(nb_hidden, train_input.size(1)).normal_(0, epsilon)
b1 = torch.empty(nb_hidden).normal_(0, epsilon)
w2 = torch.empty(nb_classes, nb_hidden).normal_(0, epsilon)
b2 = torch.empty(nb_classes).normal_(0, epsilon)

dl_dw1 = torch.empty(w1.size())
dl_db1 = torch.empty(b1.size())
dl_dw2 = torch.empty(w2.size())
dl_db2 = torch.empty(b2.size())

for k in range(1000):

    # Back-prop

    acc_loss = 0
    nb_train_errors = 0

    dl_dw1.zero_()
    dl_db1.zero_()
    dl_dw2.zero_()
    dl_db2.zero_()

    for n in range(nb_train_samples):
        x0, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, train_input[n])

        pred = x2.max(0)[1].item()
        if train_target[n, pred] < 0.5: nb_train_errors = nb_train_errors + 1
        acc_loss = acc_loss + loss(x2, train_target[n])

        backward_pass(w1, b1, w2, b2,
                      train_target[n],
                      x0, s1, x1, s2, x2,
                      dl_dw1, dl_db1, dl_dw2, dl_db2)

    # Gradient step

    w1 = w1 - eta * dl_dw1
    b1 = b1 - eta * dl_db1
    w2 = w2 - eta * dl_dw2
    b2 = b2 - eta * dl_db2

    # Test error

    nb_test_errors = 0

    for n in range(test_input.size(0)):
        _, _, _, _, x2 = forward_pass(w1, b1, w2, b2, test_input[n])

        pred = x2.max(0)[1].item()
        if test_target[n, pred] < 0.5: nb_test_errors = nb_test_errors + 1

    print('{:d} acc_train_loss {:.02f} acc_train_error {:.02f}% test_error {:.02f}%'
          .format(k,
                  acc_loss,
                  (100 * nb_train_errors) / train_input.size(0),
                  (100 * nb_test_errors) / test_input.size(0)))

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [14]:
torch.set_grad_enabled(True) #At the end, this should be set to False

#Pb set 5:
#!/usr/bin/env python

# Any copyright is dedicated to the Public Domain.
# https://creativecommons.org/publicdomain/zero/1.0/

# Written by Francois Fleuret <francois@fleuret.org>

import torch
import math

from torch import optim
from torch import Tensor
from torch import nn

######################################################################

def generate_disc_set(nb):
    input = torch.empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

#train_input, train_target = generate_disc_set(1000)
#test_input, test_target = generate_disc_set(1000)

mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

mini_batch_size = 100

######################################################################

def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss() #replace this by self macde MSE
    optimizer = optim.SGD(model.parameters(), lr = 1e-1) #replace this by self made SGD for MSE
    nb_epochs = 250

    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad() # replace this so it works with 
            loss.backward() # replace this to self made backard method
            optimizer.step() # '_'

######################################################################

def compute_nb_errors(model, data_input, data_target): #this s

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

######################################################################

def create_shallow_model():
    return nn.Sequential(
        nn.Linear(2, 128),
        nn.ReLU(),
        nn.Linear(128, 2)
    )

def create_deep_model():
    return nn.Sequential(
        nn.Linear(2, 4),
        nn.ReLU(),
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 16),
        nn.ReLU(),
        nn.Linear(16, 32),
        nn.ReLU(),
        nn.Linear(32, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 2)
    )

######################################################################

for std in [ -1, 1e-3, 1e-2, 1e-1, 1e-0, 1e1 ]:

    for m in [ create_shallow_model, create_deep_model ]:

        model = m()

        if std > 0:
            with torch.no_grad():
                for p in model.parameters(): p.normal_(0, std)

        train_model(model, train_input, train_target)

        print('std {:s} {:f} train_error {:.02f}% test_error {:.02f}%'.format(
            m.__name__,
            std,
            compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100,
            compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100
        )
        )

######################################################################

std create_shallow_model -1.000000 train_error 1.00% test_error 0.50%
std create_deep_model -1.000000 train_error 8.50% test_error 8.90%
std create_shallow_model 0.001000 train_error 1.40% test_error 1.60%
std create_deep_model 0.001000 train_error 49.60% test_error 50.00%
std create_shallow_model 0.010000 train_error 0.70% test_error 1.40%
std create_deep_model 0.010000 train_error 49.60% test_error 50.00%
std create_shallow_model 0.100000 train_error 1.00% test_error 0.50%
std create_deep_model 0.100000 train_error 50.40% test_error 50.00%
std create_shallow_model 1.000000 train_error 0.80% test_error 0.50%
std create_deep_model 1.000000 train_error 50.40% test_error 50.00%
std create_shallow_model 10.000000 train_error 0.60% test_error 0.80%
std create_deep_model 10.000000 train_error 50.40% test_error 50.00%
